## The compatibility of sQUlearn and scikit-learn

This notebook demonstrates the ability of sQUlearn to smoothly interact with scikit-learn routines. We first define a simple toy function and use the Chebychev quantum feature map to define a fidelity quantum kernel. The trainable parameters of the quantum feature map are sampled randomly. The aim in mind is to solve a regression problem using QSVR. Subsequently, we apply scikit-learn's cross validation score. Finally we demonstrate how to optimize the inherent hyperparameters of the SVR pipeline ($C$ and $\epsilon$) using GridSearchCV.

In [23]:
import numpy as np

from squlearn import Executor
from squlearn.feature_map import ChebPQC, QEKFeatureMap
from squlearn.kernel.matrix import FidelityKernel, ProjectedQuantumKernel
from squlearn.kernel.ml import QSVR

In [24]:
# Generate data set (analogously to qgpr_workflow.ipynb example)
def func(x):
    return np.sin(3 * x) * x * x


# define training and test data
x = np.linspace(-1, 1, 100)
y = func(x)
x_train = np.linspace(-1, 1, 30)
y_train = func(x_train)

In [27]:
# We can define an qml method of choice and use additional scikit-learnfunctions
# to validate the model

# 1. Step: define QML method
# set up feature map with trainable parameters being randomly initialized
# fmap = ChebPQC(num_qubits=4, num_features=1, num_layers=2, seed=42)
feature_map = QEKFeatureMap(num_qubits=4, num_features=1, num_layers=2)
# sample variational parameters of feature map randomly and pass them to kernel definition

executor = Executor('statevector_simulator')

fidelity_kernel = FidelityKernel(feature_map=feature_map, executor=executor, parameter_seed=0)
projected_kernel = ProjectedQuantumKernel(feature_map=feature_map, executor=executor, parameter_seed=0)

In [28]:
# we could also use scikit-learnroutines to optimize hyperparameters
from sklearn.model_selection import GridSearchCV

parameters = [
    {'quantum_kernel': [fidelity_kernel], 'num_qubits': [1, 2, 4], 'num_layers': [1, 2], 'some_classical_parameter': []},
    {'quantum_kernel': [projected_kernel], 'num_qubits': [1, 2, 4], 'num_layers': [1, 2], 'some_classical_parameter': []}
]

qsvc = QSVC()
grid_search = GridSearchCV(qsvc, parameters)

Cross validation score: [-18.56767443  -6.70785946   0.1486337   -6.68597084 -18.54866531]
Cross validation prediction: [ 9.64310993e-02  9.66014555e-02  9.72791580e-02  9.83079929e-02
  9.83511389e-02  9.71336504e-02  9.65991449e-02  9.64125903e-02
  9.61227076e-02  9.58176774e-02  9.55499205e-02  9.53525872e-02
 -6.63094214e-04 -7.65192760e-05  3.94521643e-04  7.95263963e-04
  1.16428252e-03  1.51700112e-03 -9.54292061e-02 -9.55475716e-02
 -9.56872863e-02 -9.58080859e-02 -9.58374549e-02 -9.56842224e-02
 -9.63483966e-02 -9.77248147e-02 -9.83438244e-02 -9.78596876e-02
 -9.70421250e-02 -9.48753480e-02]


In [9]:

# 2. Step load and use additional scikit-learn routines
from sklearn.model_selection import cross_val_score, cross_val_predict
                                                 
print("Cross validation score:", cross_val_score(QSVR, x_train.reshape(-1, 1), y_train, cv=5))
print(
    "Cross validation prediction:",
    cross_val_predict(qsvr_fqk, x_train.reshape(-1, 1), y_train, cv=5))

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [6]:
qsvr_grid.fit(x_train.reshape(-1, 1), y_train)

c:\Users\mcr\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mcr\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
AttributeError: 'NoneType' object has no attribute 'fit'

  warnings.warn(some_fits_failed_message, FitFailedWarning)
c:\Users\mcr\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan na

TypeError: Cannot clone object 'None' (type <class 'NoneType'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [7]:
from sklearn.metrics import mean_squared_error as mse

print("Best parameters:", qsvr_grid.best_params_)
print(
    "MSE with optimized hyperparameters:", mse(y_train, qsvr_grid.predict(x_train.reshape(-1, 1)))
)

Best parameters: {'C': 0.001, 'epsilon': 0.01}


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [29]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc = SVC()
grid_search = GridSearchCV(svc, parameters)

In [35]:
from squlearn.kernel import QSVC

QSVC(quantum_kernel=None)

AttributeError: 'NoneType' object has no attribute 'get_params'